Anova_tfidf_50features

In [1]:
from google.colab import drive
drive.mount('/content/drive') 

Mounted at /content/drive


In [2]:
import os 
import numpy as np 
import pandas as pd 
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier 

In [3]:
import pandas as pd
path = "/content/drive/MyDrive/newpro/threeclass_problem/ANOVA_Tfidf/data.csv"
df = pd.read_csv(path)
df.head() 

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,Level 5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,Level 5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,Level 5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",Level 5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Level 2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [4]:
df.iloc[:,1].replace(['Level 1', 'Level 2', 'Level 3', 'Level 4', 'Level 5'],
                        [1, 2, 3, 4, 5], inplace=True)

In [5]:
df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [6]:
#define bins
bins = [0,2,4,5]
#name of the group 
group_names = ['Low', 'Moderate', 'High']
df['Level '] = pd.cut(df["Level "],bins,labels = group_names) 

In [7]:
df.head(5)

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,High,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,High,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,High,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",High,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Low,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [8]:
df['Level '] = df['Level '].map({'Low': 1, 'Moderate': 2, 'High': 3}).astype(int)

In [9]:
df.head() 

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,3,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,3,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,3,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",3,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,1,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [10]:

import pandas as pd
path = "/content/drive/MyDrive/newpro/threeclass_problem/ANOVA_Tfidf/tfidf_withstopwordchangelevels_50_features.csv"
x = pd.read_csv(path)
x.head() 

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,40,41,42,43,44,45,46,47,48,49
0,0,-0.086085,-0.086085,-0.113663,-0.256451,-0.070533,-0.383901,-0.131565,-0.117292,-0.086085,...,-0.086085,-0.086085,-0.086085,-0.082814,-0.086085,-0.086085,-0.086085,-0.086085,-0.086085,-0.086085
1,1,-0.086085,-0.086085,-0.113663,-0.256451,-0.070533,-0.383901,-0.131565,-0.117292,-0.086085,...,-0.086085,-0.086085,-0.086085,-0.082814,-0.086085,-0.086085,-0.086085,-0.086085,-0.086085,-0.086085
2,2,-0.086085,-0.086085,-0.113663,-0.256451,-0.070533,-0.383901,-0.131565,-0.117292,-0.086085,...,-0.086085,-0.086085,-0.086085,-0.082814,-0.086085,-0.086085,-0.086085,-0.086085,-0.086085,-0.086085
3,3,-0.086085,-0.086085,-0.113663,-0.256451,-0.070533,-0.383901,-0.131565,-0.117292,-0.086085,...,-0.086085,-0.086085,-0.086085,-0.082814,-0.086085,-0.086085,-0.086085,-0.086085,-0.086085,-0.086085
4,4,-0.086085,-0.086085,-0.113663,-0.256451,-0.070533,-0.383901,-0.131565,-0.117292,-0.086085,...,-0.086085,-0.086085,-0.086085,-0.082814,-0.086085,-0.086085,-0.086085,-0.086085,-0.086085,-0.086085


In [11]:
x.drop(x.columns[[0]], axis=1, inplace=True)
x.head() 

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,-0.086085,-0.086085,-0.113663,-0.256451,-0.070533,-0.383901,-0.131565,-0.117292,-0.086085,-0.203472,...,-0.086085,-0.086085,-0.086085,-0.082814,-0.086085,-0.086085,-0.086085,-0.086085,-0.086085,-0.086085
1,-0.086085,-0.086085,-0.113663,-0.256451,-0.070533,-0.383901,-0.131565,-0.117292,-0.086085,-0.203472,...,-0.086085,-0.086085,-0.086085,-0.082814,-0.086085,-0.086085,-0.086085,-0.086085,-0.086085,-0.086085
2,-0.086085,-0.086085,-0.113663,-0.256451,-0.070533,-0.383901,-0.131565,-0.117292,-0.086085,-0.203472,...,-0.086085,-0.086085,-0.086085,-0.082814,-0.086085,-0.086085,-0.086085,-0.086085,-0.086085,-0.086085
3,-0.086085,-0.086085,-0.113663,-0.256451,-0.070533,-0.383901,-0.131565,-0.117292,-0.086085,-0.203472,...,-0.086085,-0.086085,-0.086085,-0.082814,-0.086085,-0.086085,-0.086085,-0.086085,-0.086085,-0.086085
4,-0.086085,-0.086085,-0.113663,-0.256451,-0.070533,-0.383901,-0.131565,-0.117292,-0.086085,-0.203472,...,-0.086085,-0.086085,-0.086085,-0.082814,-0.086085,-0.086085,-0.086085,-0.086085,-0.086085,-0.086085


In [12]:
y=df['Level '] 

**Train Test Split**

In [13]:
# Train-Test-Split is performed

from sklearn.model_selection import train_test_split

# Splitting the data into train and test
X_train, X_test, y_train, y_test = train_test_split(x, y, train_size=0.7, random_state=42)
X_train.shape, X_test.shape

((1220, 50), (524, 50))

In [14]:
print(y_train.value_counts(normalize=True)*100)
print("\n")
print(y_test.value_counts(normalize=True)*100)

2    45.327869
1    29.590164
3    25.081967
Name: Level , dtype: float64


2    45.229008
1    29.961832
3    24.809160
Name: Level , dtype: float64


**Model Building**

In [ ]:
def perceptron_classifier (X_train, y_train, X_test, y_test):
  p = Perceptron(random_state=150)
  p.fit(X_train, y_train)
  predictions_train = p.predict(X_train)
  predictions_test = p.predict(X_test)
  train_score = accuracy_score(predictions_train, y_train)
  #print("score on train data: ", train_score)
  test_score = f1_score(predictions_test, y_test, average='weighted' )
  print("Perceptron F1 score:", test_score)

In [ ]:
from sklearn.linear_model import Perceptron
perceptron_classifier(X_train, y_train, X_test, y_test) 

Perceptron F1 score: 0.3526240856057244


**Decision Tree**

In [15]:
clf_dt = DecisionTreeClassifier(max_depth=5, class_weight = "balanced")

In [16]:
clf_dt.fit(X_train,y_train)

DecisionTreeClassifier(class_weight='balanced', max_depth=5)

In [17]:
y_pred_train = clf_dt.predict(X_train)
y_pred_test = clf_dt.predict(X_test)

In [18]:
def classifcation_report_train_test(y_train, y_train_pred, y_test, y_test_pred):

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TRAIN DATA
            =========================================
            ''')
    print(classification_report(y_train, y_train_pred, digits=4))

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TEST DATA
            =========================================
            ''')
    print(classification_report(y_test, y_test_pred, digits=4))

In [19]:
scores = pd.DataFrame(columns=['Model', 'Train_Accuracy', 'Train_Recall', 'Train_Precision', 'Train_F1_Score', 
                               'Test_Accuracy', 'Test_Recall', 'Test_Precision', 'Test_F1_Score'])

def get_metrics(train_actual, train_predicted, test_actual, test_predicted, model_description, dataframe):

    train_accuracy  = accuracy_score(train_actual, train_predicted)
    train_recall    = recall_score(train_actual, train_predicted, average="weighted")
    train_precision = precision_score(train_actual, train_predicted, average="weighted")
    train_f1score   = f1_score(train_actual, train_predicted, average="weighted")
    
    test_accuracy   = accuracy_score(test_actual, test_predicted)
    test_recall     = recall_score(test_actual, test_predicted, average="weighted")
    test_precision  = precision_score(test_actual, test_predicted, average="weighted")
    test_f1score    = f1_score(test_actual, test_predicted, average="weighted")

    dataframe       = dataframe.append(pd.Series([model_description, 
                                                  train_accuracy, train_recall, train_precision, train_f1score,
                                                  test_accuracy, test_recall, test_precision, test_f1score],
                                                 index=scores.columns ), 
                                       ignore_index=True)

    return(dataframe)

In [20]:
scores = get_metrics(y_train, y_pred_train, y_test, y_pred_test, "DecisionTrees", scores)
scores

<ipython-input-19-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.292623,0.292623,0.643461,0.199634,0.272901,0.272901,0.475786,0.176044


**Random Forest**

In [21]:
clf_rf = RandomForestClassifier(class_weight = "balanced")

In [22]:
clf_rf.fit(X=X_train, y=y_train)

RandomForestClassifier(class_weight='balanced')

In [23]:
y_pred_train1 = clf_rf.predict(X_train)
y_pred_test1 = clf_rf.predict(X_test)

In [24]:
classifcation_report_train_test(y_train, y_pred_train1, y_test, y_pred_test1)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.9956    0.6260    0.7687       361
           2     0.6832    0.9946    0.8100       553
           3     0.9787    0.6013    0.7449       306

    accuracy                         0.7869      1220
   macro avg     0.8858    0.7406    0.7746      1220
weighted avg     0.8498    0.7869    0.7815      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3704    0.2548    0.3019       157
           2     0.4646    0.6920    0.5559       237
           3     0.2381    0.1154    0.1554       130

    accuracy                         0.4179       524
   macro avg     0.3577    0.3540    0.3378       524
weighted avg     0.3802    0.4179    0.3805       524



In [25]:
scores = get_metrics(y_train, y_pred_train1, y_test, y_pred_test1, "RandomForest", scores)
scores

<ipython-input-19-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.292623,0.292623,0.643461,0.199634,0.272901,0.272901,0.475786,0.176044
1,RandomForest,0.786885,0.786885,0.849775,0.781470,0.417939,0.417939,0.380168,0.380457


**AdaBoost**

In [ ]:
clf_adaboost =  AdaBoostClassifier()

In [ ]:
clf_adaboost.fit(X_train, y_train)

AdaBoostClassifier()

In [ ]:
y_pred_train2 = clf_adaboost.predict(X_train)
y_pred_test2 = clf_adaboost.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train2, y_test, y_pred_test2)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.6071    0.0942    0.1631       361
           2     0.4738    0.9656    0.6357       553
           3     0.7568    0.0915    0.1633       306

    accuracy                         0.4885      1220
   macro avg     0.6126    0.3838    0.3207      1220
weighted avg     0.5842    0.4885    0.3774      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3913    0.0573    0.1000       157
           2     0.4601    0.9494    0.6198       237
           3     0.3333    0.0308    0.0563       130

    accuracy                         0.4542       524
   macro avg     0.3949    0.3458    0.2587       524
weighted avg     0.4080    0.4542    0.3243       524



In [ ]:
scores = get_metrics(y_train, y_pred_train2, y_test, y_pred_test2, "Adaboost", scores)
scores

<ipython-input-21-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.468852,0.468852,0.650446,0.321239,0.448473,0.448473,0.324079,0.287301
1,RandomForest,0.786885,0.786885,0.848701,0.781575,0.412214,0.412214,0.377874,0.377519
2,Adaboost,0.488525,0.488525,0.584239,0.377358,0.454198,0.454198,0.408048,0.324284


**Gradient Boosting Classifier**

In [ ]:
clf_gbm = GradientBoostingClassifier()

In [ ]:
clf_gbm.fit(X=X_train, y=y_train)

GradientBoostingClassifier()

In [ ]:
y_pred_train3 = clf_gbm.predict(X_train)
y_pred_test3 = clf_gbm.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train3, y_test, y_pred_test3)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.8952    0.3075    0.4577       361
           2     0.5284    0.9910    0.6893       553
           3     1.0000    0.1928    0.3233       306

    accuracy                         0.5885      1220
   macro avg     0.8079    0.4971    0.4901      1220
weighted avg     0.7552    0.5885    0.5290      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.4000    0.1019    0.1624       157
           2     0.4559    0.9156    0.6087       237
           3     0.2500    0.0154    0.0290       130

    accuracy                         0.4485       524
   macro avg     0.3686    0.3443    0.2667       524
weighted avg     0.3881    0.4485    0.3312       524



In [ ]:
scores = get_metrics(y_train, y_pred_train3, y_test, y_pred_test3, "GBM", scores)
scores

<ipython-input-21-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.468852,0.468852,0.650446,0.321239,0.448473,0.448473,0.324079,0.287301
1,RandomForest,0.786885,0.786885,0.848701,0.781575,0.412214,0.412214,0.377874,0.377519
2,Adaboost,0.488525,0.488525,0.584239,0.377358,0.454198,0.454198,0.408048,0.324284
3,GBM,0.588525,0.588525,0.755233,0.528979,0.448473,0.448473,0.388061,0.331167


**SVM Model**

In [26]:
linear_svm = SVC(kernel='linear', C=1, class_weight = "balanced")

In [27]:
linear_svm.fit(X=X_train, y= y_train)

SVC(C=1, class_weight='balanced', kernel='linear')

In [28]:
y_pred_train4 = linear_svm.predict(X_train)
y_pred_test4 = linear_svm.predict(X_test)

In [29]:
classifcation_report_train_test(y_train, y_pred_train4, y_test, y_pred_test4)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.4021    0.1080    0.1703       361
           2     0.4835    0.2116    0.2943       553
           3     0.2690    0.7745    0.3993       306

    accuracy                         0.3221      1220
   macro avg     0.3848    0.3647    0.2880      1220
weighted avg     0.4056    0.3221    0.2840      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3654    0.1210    0.1818       157
           2     0.4595    0.2152    0.2931       237
           3     0.2632    0.7308    0.3870       130

    accuracy                         0.3149       524
   macro avg     0.3627    0.3557    0.2873       524
weighted avg     0.3826    0.3149    0.2830       524



In [30]:
scores = get_metrics(y_train, y_pred_train4, y_test, y_pred_test4, "SVM", scores)
scores

<ipython-input-19-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.292623,0.292623,0.643461,0.199634,0.272901,0.272901,0.475786,0.176044
1,RandomForest,0.786885,0.786885,0.849775,0.781470,0.417939,0.417939,0.380168,0.380457
2,SVM,0.322131,0.322131,0.405592,0.283970,0.314885,0.314885,0.382572,0.283047


**KNN**

In [ ]:
knn = KNeighborsClassifier(n_neighbors=7)

In [ ]:
knn.fit(X=X_train, y=y_train)

KNeighborsClassifier(n_neighbors=7)

In [ ]:
y_pred_train5 = knn.predict(X_train)
y_pred_test5 = knn.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train5, y_test, y_pred_test5)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.4869    0.3601    0.4140       361
           2     0.5772    0.4123    0.4810       553
           3     0.2975    0.5425    0.3843       306

    accuracy                         0.4295      1220
   macro avg     0.4539    0.4383    0.4264      1220
weighted avg     0.4803    0.4295    0.4369      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3445    0.2611    0.2971       157
           2     0.4590    0.3544    0.4000       237
           3     0.2658    0.4538    0.3352       130

    accuracy                         0.3511       524
   macro avg     0.3564    0.3565    0.3441       524
weighted avg     0.3768    0.3511    0.3531       524



In [ ]:
scores = get_metrics(y_train, y_pred_train5, y_test, y_pred_test5, "KNN", scores)
scores

<ipython-input-21-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.468852,0.468852,0.650446,0.321239,0.448473,0.448473,0.324079,0.287301
1,RandomForest,0.786885,0.786885,0.848701,0.781575,0.412214,0.412214,0.377874,0.377519
2,Adaboost,0.488525,0.488525,0.584239,0.377358,0.454198,0.454198,0.408048,0.324284
3,GBM,0.588525,0.588525,0.755233,0.528979,0.448473,0.448473,0.388061,0.331167
4,SVM,0.462295,0.462295,0.487195,0.315289,0.452290,0.452290,0.322121,0.308767
5,KNN,0.429508,0.429508,0.480328,0.436920,0.351145,0.351145,0.376773,0.353100


**GNB**

In [ ]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()

In [ ]:
nb.fit(X=X_train, y=y_train)

GaussianNB()

In [ ]:
y_pred_train6 = nb.predict(X_train)
y_pred_test6 = nb.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train6, y_test, y_pred_test6)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.4000    0.0166    0.0319       361
           2     0.4596    0.9765    0.6250       553
           3     0.4333    0.0425    0.0774       306

    accuracy                         0.4582      1220
   macro avg     0.4310    0.3452    0.2448      1220
weighted avg     0.4354    0.4582    0.3122      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.5000    0.0255    0.0485       157
           2     0.4562    0.9662    0.6198       237
           3     0.2143    0.0231    0.0417       130

    accuracy                         0.4504       524
   macro avg     0.3902    0.3383    0.2366       524
weighted avg     0.4093    0.4504    0.3052       524



In [ ]:
scores = get_metrics(y_train, y_pred_train6, y_test, y_pred_test6, "GNB", scores)
scores

<ipython-input-21-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.468852,0.468852,0.650446,0.321239,0.448473,0.448473,0.324079,0.287301
1,RandomForest,0.786885,0.786885,0.848701,0.781575,0.412214,0.412214,0.377874,0.377519
2,Adaboost,0.488525,0.488525,0.584239,0.377358,0.454198,0.454198,0.408048,0.324284
3,GBM,0.588525,0.588525,0.755233,0.528979,0.448473,0.448473,0.388061,0.331167
4,SVM,0.462295,0.462295,0.487195,0.315289,0.452290,0.452290,0.322121,0.308767
5,KNN,0.429508,0.429508,0.480328,0.436920,0.351145,0.351145,0.376773,0.353100
6,GNB,0.458197,0.458197,0.435364,0.312152,0.450382,0.450382,0.409295,0.305174
